In [14]:
from datasets import load_dataset, Dataset, concatenate_datasets
import pandas as pd
import csv
import pandas as pd


# with open('./verbalist/datasets/ficbook/fic_anx_gen 3.csv', 'r', encoding='utf-8') as file:
#     reader = csv.DictReader(file)
#     df = pd.DataFrame(reader)
dataset = load_dataset("dim/ficbook_raw")
dataset = dataset['train']
dataset

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--ficbook_raw-457acaea4b4f1830/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'author', 'title', 'link', 'description', 'tag', 'likes', 'date', 'review', 'format', 'text', 'rating', 'status', 'parts'],
    num_rows: 114411
})

In [9]:
# dataset = Dataset.from_pandas(df)

In [12]:
# dataset.push_to_hub("dim/ficbook_raw", private=True)

In [15]:
dataset = dataset.train_test_split(test_size=100_000, seed=42)
dataset = dataset['test']
dataset

Dataset({
    features: ['id', 'author', 'title', 'link', 'description', 'tag', 'likes', 'date', 'review', 'format', 'text', 'rating', 'status', 'parts'],
    num_rows: 100000
})

In [16]:
# dataset.push_to_hub("dim/ficbook_raw_100k", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/50 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/50 [00:00<?, ?ba/s]